# Reto 3

## Miguel Ortiz y Andrea Castiella


In [25]:
import numpy  as np
import pandas as pd
import OurUtils as our
from sklearn.preprocessing import MinMaxScaler

In [26]:
# Read data
X_full = pd.read_csv('Reto 3-Dataset/reto3_trainX.csv', header=0, low_memory=False)
Y_full = pd.read_csv('Reto 3-Dataset/reto3_trainY.csv', header=0, low_memory=False)

print(X_full.head())
print(Y_full.head())

   altitud  azimut  inclinacion  DH_agua  DV_agua  DH_camino  sombra_9am  \
0     3351     354           19      450       95       1064         185   
1     2995      84            7      481       30       5154         231   
2     2884      72           25      210      -45        466         239   
3     3139      76           11      301       58       1368         234   
4     3018     312           17       30       10       1719         172   

   sombra_12pm  sombra_3pm  DH_fuego  ...  t31  t32  t33  t34  t35  t36  t37  \
0          203         153       711  ...    0    0    0    0    0    0    0   
1          227         129      5488  ...    0    0    0    0    0    0    0   
2          183          60      2123  ...    0    0    0    0    0    0    0   
3          220         117      3282  ...    0    0    1    0    0    0    0   
4          225         193      1961  ...    0    0    0    0    0    0    0   

   t38  t39  t40  
0    0    1    0  
1    0    0    0  
2    

## Split dataset
Train split: 80%

Validation split: 20%

In [27]:
# Split into train and validation
valid_size = 0.2
seed = 1234
X_train, Y_train, X_valid, Y_valid = \
   our.single_stratified_split(X_full, Y_full, test_size=valid_size, random_state=seed)

## Unbalanced class problem

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

over = SMOTE(random_state=seed, sampling_strategy={3:10000 , 4:10000 , 5: 10000, 6:10000 , 7:10000})
X_train, Y_train = over.fit_resample(X_train, Y_train)

under = RandomUnderSampler(random_state=seed, sampling_strategy={1:10000 , 2:10000})
X_train, Y_train = over.fit_resample(X_train, Y_train)

In [4]:
# X_train = X_train.iloc[:,[23, 0, 13, 5, 25, 35, 51]]
# X_valid = X_valid.iloc[:,[23, 0, 13, 5, 25, 35, 51]]


## Scale and transform features

In [28]:
## train
X, Y, scaler, pca, transformer = our.our_scale_transform_features(X_train,Y_train)

## val
X_pred, Y_true, scaler, pca, transformer = our.our_scale_transform_features(X_valid,Y_valid, scaler=scaler, transform=transformer)

# # scale features
# scaler = MinMaxScaler()
# X = scaler.fit_transform(X_train)

# # pca
# pca_bool = False

# if pca_bool:
#     X, pca = our.our_PCA(X)

# # apply some preproccesing
# procc_type = "transform"

# if procc_type == "poly":
#     poly = PolynomialFeatures(2)
#     X = poly.fit_transform(X)

# if procc_type == "transform":
#     transformer = FunctionTransformer(np.log1p, validate=True)
#     X = transformer.fit_transform(X)

# # get Y values
# Y = Y_train.values.ravel()

# PCA

In [6]:
# X_proy, pca = our.our_PCA(X)
# num_of_pc = len(pca.singular_values_)
# strTitle = '\n First %d principal components' %(num_of_pc)
# print(strTitle)
# print(pca.components_.T)
# strTitle = '\n First %d singular values ' %(num_of_pc)
# print(strTitle)
# print(pca.singular_values_.T)
# strTitle = '\n Explained Variance Ratio'
# print(strTitle)
# print(pca.explained_variance_ratio_.T)
# strTitle = '\n Accumulated Explained Variance Ratio'
# print(strTitle)
# print(np.cumsum(pca.explained_variance_ratio_.T))
# X = X_proy

(92961, 4)

 First 4 principal components
[[ 0.54657452  0.07371693  0.04353025  0.41961752]
 [-0.32583588  0.08312472 -0.1163221   0.05576592]
 [ 0.66396056  0.07417547  0.08131946  0.1662541 ]
 [-0.36001774 -0.05669879  0.01621619  0.88298115]
 [-0.0573745  -0.5166215   0.83708605 -0.0244898 ]
 [-0.13463679  0.84361369  0.51143895 -0.00989523]
 [-0.05667427 -0.01890801 -0.12415964  0.11321439]]

 First 4 singular values 
[96.83252594 71.72809213 66.34651326 62.59299545]

 Explained Variance Ratio
[0.32648582 0.17914321 0.1532703  0.1364185 ]

 Accumulated Explained Variance Ratio
[0.32648582 0.50562904 0.65889934 0.79531784]


## Multiclass. OvO vs OvR

In [34]:
# Multiclass
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

n_estimators = 150
max_leaf_nodes = 20
max_depth = 2

#weights = {1:.25, 2:.25, 3:.7, 4:1 , 5:.7 ,6:.7 , 7:.7 }
base_clf = RandomForestClassifier(n_estimators=n_estimators,
                                max_depth=max_depth)#, class_weight=weights)
                                #max_leaf_nodes=max_leaf_nodes)



base_clf.fit(X,Y)

# Train OvO 
OvO_clf = OneVsOneClassifier(base_clf)
OvO_clf.fit(X,Y)

# Train OvR 
OvR_clf = OneVsRestClassifier(base_clf)
OvR_clf.fit(X,Y)

# Check
strlog = "OvO produces %d classifiers" %(len(OvO_clf.estimators_))
print(strlog)
strlog = "OvR produces %d classifiers" %(len(OvR_clf.estimators_))
print(strlog)


OvO produces 21 classifiers
OvR produces 7 classifiers


## Validation

In [35]:
# # Feature engineering, selection and rescaling to [0,1]
# X_pred = scaler.transform(X_valid)
# # X_pred = poly.fit_transform(X_pred)
# X_pred = transformer.fit_transform(X_pred)
# # PCA
# #X_pred = pca.transform(X_pred)
# Y_true = Y_valid.values.ravel()

# predict
Y_pred = base_clf.predict_proba(X_pred)

# predict
Y_pred_OvO = OvO_clf.predict(X_pred)
Y_pred_OvR = OvR_clf.predict(X_pred)


## Confusion Matrix

To check the performance of our model, we use Confusion Matrix as metric. This give us a general idea of how good is the model and let us compare it with the multiple combinations of feature that we have available.

In [36]:
# Performance metrics

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import cohen_kappa_score

conf_mat_OvO = confusion_matrix(Y_true,Y_pred_OvO)
hits_OvO = np.trace(conf_mat_OvO)
conf_mat_OvR = confusion_matrix(Y_true,Y_pred_OvR)
hits_OvR = np.trace(conf_mat_OvR)

# Print out
print("\nOvO confusion matrix:\n")
print(conf_mat_OvO)
print("\n")
print( "OvO Hits  = %d"%(hits_OvO) ) 
print( "OvO Fails = %d"%(Y_true.shape[0]-hits_OvO) )
print( "OvO Accuracy = %f"%(hits_OvO/(Y_true.shape[0])))
print("\nOvR confusion matrix:\n")
print( "OvO Cohen Accuracy = %f"%(cohen_kappa_score(Y_true, Y_pred_OvO)))
print(conf_mat_OvR)
print( "\n")
print( "OvR Hits  = %d"%(hits_OvR) ) 
print( "OvR Fails = %d"%(Y_true.shape[0]-hits_OvR) )
print( "OvR Accuracy = %f"%(hits_OvR/Y_true.shape[0]))
print( "OvR Cohen Accuracy = %f"%(cohen_kappa_score(Y_true, Y_pred_OvO)))



OvO confusion matrix:

[[ 4094  4380     0     0     0     0     0]
 [ 1066 10170    17     3     0    76     0]
 [    0   610   262   252     0   306     0]
 [    0     0     1   102     0     7     0]
 [    0   380     0     0     0     0     0]
 [    0   299    82    56     0   258     0]
 [  747    73     0     0     0     0     0]]


OvO Hits  = 14886
OvO Fails = 8355
OvO Accuracy = 0.640506

OvR confusion matrix:

OvO Cohen Accuracy = 0.371103
[[ 2057  6348     0     0     0     0    69]
 [  522 10733     4    13     0    58     2]
 [    0   652    76   427     0   275     0]
 [    0     1     0   109     0     0     0]
 [    0   380     0     0     0     0     0]
 [    0   330    32   163     0   170     0]
 [  198   413     0     0     0     0   209]]


OvR Hits  = 13354
OvR Fails = 9887
OvR Accuracy = 0.574588
OvR Cohen Accuracy = 0.371103
